In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!head "data_vn.txt"

__label__tot Tỷ lệ tốt Giá / Dịch vụ.Lời khuyên tốt cho Vườn quốc gia và Làng. Tốt
__label__trung_binh Trang thiết bị vệ sinh hơi cũ. Vệ sinh sạch sẽ, nhân viên thân thiện, địa điểm tốt..  Một khách sạn sạch sẽ, thoải mái, dễ chịu, địa điểm tốt
__label__xuat_sac Nhân viên thân thiện.Hữu ích để đặt tour du lịch.Phòng sạch sẽ và yên tĩnh..Vị trí hoàn hảo cho hang động công viên quốc gia Ke Bang
__label__kem Quá ồn ào . Thất vọng
__label__tot Nơi tốt đẹp để có được xung quanh nhân viên thực sự chăm sóc bạn.Ở lại hoàn hảo, siêu nhân viên
__label__xuat_sac - Phòng tốt / Giường - Nhân viên rất đẹp và hữu ích - Tuyệt vời - Không khí thân thiện.Một nơi đáng yêu để ở lại
__label__tot Giường khó nhất Ive ngủ vào bữa sáng giá trị nhân viên hữu ích.Giá trị tốt
__label__xuat_sac Nơi rất đẹp!Gia đình rất thân thiện!.Ở lại rất tốt đẹp
__label__xuat_sac Todco Pertact.Chúng tôi đã được hai ngày và nó đã hoàn hảo.Họ đã giúp chúng tôi trong mọi thứ chúng tôi có nhu cầu
__label__xuat_sac Bãi biển đẹp . Mộ

In [ ]:
!pip install underthesea

     |████████████████████████████████| 7.5 MB 14.4 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 3.1 MB 49.2 MB/s 
     |████████████████████████████████| 743 kB 51.0 MB/s 
     |████████████████████████████████| 235 kB 69.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.4 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 53.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=c03532eaf4f6ac0a1627bf95a5087aca5b743d6d3dcdeebd6cb6b6cfb365137f
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import time

import numpy as np
import regex
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from underthesea import word_tokenize

VN_DATA_DIRECT = 'data_vn.txt'

# with open(f"data/stop_word_vn.txt", encoding="utf8") as f:
#     stop_word = f.read().splitlines()


def remove_character_not_ness(document):
    document = regex.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', ' ', document)
    document = regex.sub(r'[^\D]', ' ', document)
    document = regex.sub(r'\s+', ' ', document).strip()
    return document


def no_accent_vietnamese(s):
    s = regex.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = regex.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = regex.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = regex.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = regex.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = regex.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = regex.sub(r'[ìíịỉĩ]', 'i', s)
    s = regex.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = regex.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = regex.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = regex.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = regex.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = regex.sub(r'[Đ]', 'D', s)
    s = regex.sub(r'[đ]', 'd', s)
    return s


def remove_stopwords(document):
    words = []
    for word in document.strip().split():
        if word not in stop_word:
            words.append(word)
    return ' '.join(words)


def text_preprocess(document):
    document = document.lower()
    document = word_tokenize(document, format="text")
    document = remove_character_not_ness(document)
    #document = remove_stopwords(document)
    return document


text = []
label = []


def load_training_data(
        vn_directory
):
    with open(f"{vn_directory}", encoding="utf8") as f:
        data_set = f.read().splitlines()
        for data in data_set:
            document = data
            tmp = document.split(" ", 1)
            tmp[1] = text_preprocess(tmp[1])
            label.append(tmp[0])
            text.append(tmp[1])
            s = no_accent_vietnamese(tmp[1]).replace('_', ' ')
            if tmp[1] != s:
                label.append(tmp[0])
                text.append(s)


load_training_data(VN_DATA_DIRECT)

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=0.1, random_state=42, shuffle=True)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 6),
                                              max_df=0.8,
                                              )),
                     ('tfidf', TfidfTransformer(use_idf=False, sublinear_tf=True)),
                     ('clf', LinearSVC())
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

y_pred = text_clf.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))

y_pred = text_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))


Done training SVM in 61.73260760307312 seconds.
SVM, Accuracy = 0.6961386573058359
                     precision    recall  f1-score   support

       __label__kem       0.61      0.46      0.52       214
   __label__rat_kem       1.00      0.19      0.32        48
       __label__tot       0.68      0.82      0.74      2072
__label__trung_binh       0.70      0.54      0.61       951
  __label__xuat_sac       0.74      0.67      0.70      1273

           accuracy                           0.70      4558
          macro avg       0.75      0.54      0.58      4558
       weighted avg       0.70      0.70      0.69      4558



In [14]:

test = ['Bách béo bị đói ăn', 'tàm tạm', 'Tốt', 'thất vỌng', 'tệ', 'Phong kha là dep.', 'Tôi thực sự ấn tượng '
                                                                               'với phong cách làm '
                                                                               'việc của Chudu24. '
                                                                               'Nhiệt tình, '
                                                                               'hiệu quả. Tôi cũng '
                                                                               'gửi lời cám ơn tới '
                                                                               'bộ phận chăm sóc '
                                                                               'khách hàng Chudu24 '
                                                                               'nói chung và bạn Lai '
                                                                               'NV chăm sóc khách '
                                                                               'hàng nói riêng. Chúc '
                                                                               'Quý Công ty Thành '
                                                                               'công hơn nữa trong '
                                                                               'tương lai. . Cám ơn '
                                                                               'Chudu24']
for i in range(len(test)):
    test[i] = text_preprocess(test[i])
label = text_clf.predict(test)
print('Predict label:', list(label_encoder.inverse_transform(label)))

Predict label: ['__label__trung_binh', '__label__trung_binh', '__label__tot', '__label__kem', '__label__rat_kem', '__label__tot', '__label__tot']
